# CSMC

Testing that the CSMC still appropriately filters while maintaining the immortal trajectory at the $0^{\mathrm{th}}$ index

In [3]:
from model.lgssm import LGSSMModel

import jax.random as jr
import jax.numpy as jnp
from jax.scipy.stats import multivariate_normal


from dynamax.linear_gaussian_ssm import lgssm_filter
from dynamax.linear_gaussian_ssm import LinearGaussianSSM as dmax_LGSSM

import matplotlib.pyplot as plt

from feynman_kac.protocol import FeynmacKac, PFConfig
from feynman_kac.bootstrap import BootstrapParticleFilter, ConditionalBPF
from utils.datasets import load_dataset

In [14]:

def exact_filter(data):
    
    # get true filter means using Kalman filter
    dmax_lgssm = dmax_LGSSM(data.train_states.shape[-1], data.train_data[0].shape[-1])
    params, _ = dmax_lgssm.initialize(
        jr.PRNGKey(0),
        initial_mean=data.params['m1'],
        initial_covariance=data.params['Q1'],
        dynamics_weights=data.params['A'],
        dynamics_covariance=data.params['Q'],
        emission_weights=data.params['C'],
        emission_covariance=data.params['R']
    )
    x, y = data.train_states[0], data.train_data[0][0]
    lgssm_posterior = lgssm_filter(params, y)
    print("Exact filter means shape: ", lgssm_posterior.filtered_means.shape)
    return lgssm_posterior


def bpf(data, N: int):

    # parametrise model with the true parameters for now
    lgssm = LGSSMModel(data.params)
    
    # construct bootstrap particle filter
    bpf = BootstrapParticleFilter(
        model=lgssm,
        config=PFConfig(
            N=N,
            resample_scheme='multinomial',
            ess_threshold=0.5,
            key=jr.PRNGKey(42)
        )
    )

    # run filter
    outs = bpf.filter(
        T=data.train_data[0].shape[1],
        obs=data.train_data[0][0]  # just using the first sequence
    )
    print("BPF particles shape: ", outs.particles.shape)
    print("BPF weights shape: ", outs.weights.shape)
    print("BPF logZ_hat: ", outs.logZ_hat)
    return outs


def csmc_bpf(data, N: int):

    lgssm = LGSSMModel(data.params)
    T = data.train_data[0].shape[1]
    obs = data.train_data[0][0]  # just use the first sequence

    # run a single particle through BPF to get immortal trajectory
    bpf = BootstrapParticleFilter(
        model=lgssm,
        config=PFConfig(
            N=1,
            resample_scheme='multinomial',
            ess_threshold=0.5,
            key=jr.PRNGKey(42)
        )
    )
    bpf_outs = bpf.filter(T=T, obs=obs)
    x_imm = bpf_outs.particles[:, 0]
    print(x_imm.shape)
    
    # same config as regular BPF
    csmc = ConditionalBPF(
        model=lgssm,
        config=PFConfig(
            N=N,
            resample_scheme='multinomial',
            ess_threshold=0.5,
            key=jr.PRNGKey(42)
        )
    )
    csmc_outs = csmc.csmc(T=T, obs=obs, x_imm=x_imm)
    
    print("CSMC particles shape: ", csmc_outs.particles.shape)
    print("CSMC weights shape: ", csmc_outs.weights.shape)
    print("CSMC logZ_hat: ", csmc_outs.logZ_hat)
    return x_imm, csmc_outs 


In [5]:
data = load_dataset('linear_small', seed=0)
lgssm_posterior = exact_filter(data)
x, y = data.train_states[0], data.train_data[0][0]

Exact filter means shape:  (300, 1)


In [17]:
N = 50
print(f"N: {N:<-100}")
outs = bpf(data, N)
print("\n")
x_imm, csmc_outs = csmc_bpf(data, N)

N: 50                                                                                                  
BPF particles shape:  (300, 50, 1)
BPF weights shape:  (300, 50)
BPF logZ_hat:  -751.8474731445312


(300, 1)
CSMC particles shape:  (300, 50, 1)
CSMC weights shape:  (300, 50)
CSMC logZ_hat:  -748.234619140625


In [21]:
csmc_outs.ancestors[:, 0]

Array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtyp

In [20]:
csmc_outs.particles[:, 0, :] == x_imm

Array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [